# Extract, Transform, Load (ETL)
---
The purpose of this Jupyter Notebook is to extract data and storing it in a SQLite database within this package. The codes below show the steps in this process:
- Data pulling <i>['Starbucks', 'Dunkin' Donuts', 'Think Coffee', 'Joe Coffee', 'Gregorys Coffee', 'Birch Coffee']</i> located around NYC
    - <b>NYC location -</b> longitude: -73.99429321289062, latitude: 40.70544486444615
    - Includes these areas: Brooklyn, the Bronx, Manhattan, Queens and Staten Island
    - Using search radius of 40000 (meters) or ~25 miles
- ETL - for only needed data that will be used to store in the database
- Configure database
    - Delete (if exists) and create a new database coffee_chains.sqlite and the table
    - Load data into the database

#### Note that this is not subjected to only coffee finding but can be any other search terms

In [ ]:
# Import dependencies
from api_key import api_key
from jsonschema import validate
import json
import requests
import pandas as pd

In [ ]:
# Set up Yelp API constants
API_HOST = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {
    'Authorization': 'bearer %s' % api_key
}

# Schema for comparing in checking before extraction
schema = {
    'alias': '',
    'categories': [],
    'coordinates': {},
    'display_phone': '',
    'distance': 0.00,
    'id': 'string',
    'image_url': '',
    'is_closed': True,
    'location': {},
    'name': '',
    'phone': '',
    'price': '',
    'rating': 0.0,
    'review_count': 0,
    'transaction': [],
    'url': ''
}

### Custom functions for the ETL steps

In [ ]:
# Simple request function for bussiness search endpoint from Yelp API
def request(term = '', loc = '', offsets = 200, rad = 10000):
    data = []
    
    for offset in range(0, offsets, 50):
        params = {
            'term': term.replace(' ', '+'),
            'location': loc.replace(' ', '+'),
            'limit': 50,
            'offset': offset,
            'radius': rad
        }

        # Send the request
        response = requests.get(API_HOST, headers = HEADERS, params = params)

        # Verify the response and return None if error returned else return the json data
        if response.status_code == 200:            
            data += response.json()['businesses']
        else:
            return None

    return data

# Function to verify the predefined schema on what we should be expecting before extracting
def verify_schema(data = None):
    
    # Verify the object entered before extraction
    try:
        validate(instance=data, schema=schema)
        return True
    except:
        return False

# Return the json to DF for cleanining
def json_to_dataframe(data = None):
    try:
        if not verify_schema(data[0]):
            return pd.DataFrame({'error': ["{'error': 'SCHEMA VALIDATION ERROR'}"]})
        else:
            return pd.DataFrame(data)
    except TypeError:
        return pd.DataFrame({'error': ["{'error': 'OBJECT INPUT ERROR'}"]})

# Function to extract the id, name, price, rating, review_count, location (address 1, address 2, address 3, city, 
# state, zip_code), coordinates (latitude and longtitude), and phone into a DataFrame
def cleaned_yelp_dataframe(df, name, filter_by_arr = ['']):
    # Create a copy of the df to work with
    clean_df = df.copy()
    
    # Sometimes random result return not matching the criteria, filter those out by the name
    key_search = [name.lower()]
    clean_df['name'] = clean_df.name.apply(lambda x: x.lower())
    clean_df = clean_df.loc[clean_df['name'].isin(key_search)]
    
    # Normalizing the coordinates and location columns with nested dictionary
    clean_df[['latitude', 'longitude']] = pd.json_normalize(clean_df['coordinates'])
    clean_df[[
        'address1', 
        'address2', 
        'address3', 
        'city', 
        'zip', 
        'country', 
        'state', 
        'display_address'
    ]] = pd.json_normalize(clean_df['location'])
    
    # Drop off the columns no longer needed
    clean_df = clean_df.drop(columns = [
        'alias', 'is_closed', 'categories', 'coordinates', 'transactions', 'location',
        'phone', 'distance', 'display_address'        
    ])
    
    # Add price point column, fill na, and convert to int
    clean_df['price'] = clean_df['price'].fillna('')
    clean_df['price_point'] = clean_df['price'].str.len()
    clean_df['price_point'] = clean_df['price_point'].fillna(0)
    clean_df['price_point'] = clean_df['price_point'].astype('int')
    
    # Reorganize the df for easy viewing
    clean_df = clean_df[[
        'id', 'name', 'review_count', 'rating', 'price', 'price_point', 'display_phone', 'url', 'image_url', 'address1', 
        'address2', 'address3', 'city', 'state', 'zip', 'country', 'latitude', 'longitude'
    ]]
    
    # Filter out just stores found within NYC
    key_search = filter_by
    clean_df = clean_df.loc[clean_df['city'].isin(key_search)]
    
    # Replace any "None" values from address2 and address3 to ""
    clean_df['address2'] = clean_df['address2'].fillna('')
    clean_df['address3'] = clean_df['address3'].fillna('')
    
    # Proper casing for the name and city
    clean_df['name'] = clean_df['name'].str.title()

    # Take only rows that do not have nan for coordinates
    clean_df = clean_df[clean_df['latitude'].notna()]

    return clean_df

### Perform the ETL into the database

In [ ]:
# Using for loop to pull the requests, clean, and then combine to export out into one single json file and add to database
search_locations = ['nyc', 'bronx, NY', 'queens, NY', 'staten island, NY', 'brooklyn, NY']
search_terms = ['Starbucks', 'Dunkin\' Donuts', 'Tim Hortons', 'Think Coffee', 'Joe Coffee', 'Gregorys Coffee', 'Birch Coffee']
filter_by = ['New York', 'Brooklyn', 'Bronx', 'Manhattan', 'Queens', 'Staten Island']
dfs = []

# Loop through to append the output cleaned df of each coffee chain for the merging using the defined functions from above
for loc in search_locations:
    for term in search_terms:
        response_data = request(term, loc, offsets = 200)
        response_data = json_to_dataframe(response_data)
        cleaned_response_data = cleaned_yelp_dataframe(response_data, term, filter_by)
        dfs.append(cleaned_response_data)

# Now perform the merge
df_merged = pd.concat(dfs)
# Count how many before dropping duplicates and count duplicates
print(df_merged.shape[0])
print(df_merged.groupby(df_merged.columns.tolist(),as_index=False).size())

# Drop any duplicates
df_merged = df_merged.drop_duplicates()
# Count how many after dropping duplicates
print(df_merged.shape[0])

In [ ]:
# Quick check/reveview the merged DF for reviewing
df_merged.head(20)

In [ ]:
# Print out the dtypes for the DF
df_merged.dtypes

In [ ]:
# Export out into csv and json in addition to adding it to a SQLite database
df_merged.to_csv('../static/dataset/merged.csv', index = False)
df_merged.to_json('../static/dataset/merged.json', orient = 'records', indent = 4)

In [ ]:
# Import dependencies for handling the database
from os import path, remove
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session

In [ ]:
# Setup the db path
db_path = '../coffee_chains.sqlite'

# Delete the existing database if it exists
if path.exists(db_path):
    remove(db_path)

In [ ]:
# Setup the engine and connect the database
engine = create_engine(f'sqlite:///{db_path}')
conn = engine.connect()

In [ ]:
# Append the DF to the shop table created
df_merged.to_sql(name = 'shops', con = engine, if_exists = 'replace', index = False)

In [ ]:
# Quick check to see if things got appended correctly
session = Session(bind = engine)

In [ ]:
session.execute(text('SELECT * FROM shops')).fetchone()

In [ ]:
# Close out of the session and engine
session.close()
engine.dispose()